# How to run Streamlit app from Colab notebook

This tutorial has the sample code to run a Streamlit app from Colab notebook.

### References

- https://www.google.com/search?q=launch+streamlit+app+from+Google+colab&oq=launch+streamlit+app+from+Google+colab&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIGCAEQRRg80gEJMTE2NzNqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:df17f4e1,vid:CzG1_S8mcFc,st:0

- https://alphasec.io/run-a-streamlit-app-with-google-colab-notebook/


- https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399


- https://medium.com/@yash.kavaiya3/running-streamlit-code-in-google-colab-involves-a-few-steps-c43ea0e8c0d9



### How to run
To open the notebook in Colab, please click on the badge here:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datariders/tutorials/blob/main/aws-bedrock-anthropic-polly-streamlit/aws_bedrock_anthropic_polly_streamlit.ipynb)

In [ ]:
!pip3 install -q boto3 streamlit
!npm install localtunnel

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [3]:
AWS_ACCESS_KEY_ID = "<aws_access_key_id>"
AWS_SECRET_ACCESS_KEY = "<aws_secret_access_key>"

In [ ]:
%%writefile app.py
import boto3
import json
import os
from botocore.exceptions import ClientError
import streamlit as st



MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Set up the Amazon Bedrock client
bedrock_runtime_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1',  # Replace with your AWS region
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
)
print(" bedrock_runtime_client: ", bedrock_runtime_client)

# Set up the Amazon Polly client
polly_client = boto3.client(
    service_name='polly',
    region_name='us-east-1',  # Replace with your AWS region
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
)
print(" polly_client: ", polly_client)

# Specify the Claude 3.5 Sonnet model ID
model_id = MODEL_ID
print(" model_id: ", model_id)



def generate_speech(text, voice_id='Joanna'):
    try:
        # Call Polly to synthesize the text into speech
        response = polly_client.synthesize_speech(
            Text=text,
            OutputFormat='mp3',  # You can also use 'ogg_vorbis' or 'pcm'
            VoiceId=voice_id
        )

        # Save the speech to an MP3 file
        audio_file = "output_speech.mp3"
        with open(audio_file, 'wb') as file:
            file.write(response['AudioStream'].read())

        return audio_file
    except ClientError as e:
        st.error(f"Polly error: {e}")
        return None


st.title("MathNEXT: Personal Math Tutor")

user_query = st.text_input("Enter your query:")
if user_query:
    print(" user_query: ", user_query)
    if user_query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        exit()

    try:
        # Prepare the request body
        request_body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 300,
            "temperature": 0.7,
            "messages": [
                {"role": "user", "content": user_query}
            ]
        })
        print(" request_body: ", request_body)

        # Invoke the Bedrock model
        response = bedrock_runtime_client.invoke_model(
            modelId=model_id,
            body=request_body
        )
        print(" response: ", response)

        # Parse the response
        response_body = json.loads(response['body'].read())
        output = response_body['content'][0]['text']
        print(" response_body: ", response_body)

        # Display Bedrock model response
        st.write("MathNEXT response")
        st.write(output)

        # Generate speech from the Bedrock response
        audio_file = generate_speech(output)
        print(" audio_file: ", audio_file)

        if audio_file:
            # Provide a link to download the speech
            with open(audio_file, 'rb') as file:
                st.audio(file.read(), format="audio/mp3")

            st.download_button(
                label="Download Speech",
                data=open(audio_file, 'rb'),
                file_name=audio_file,
                mime='audio/mp3'
            )
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        st.error(f"Error: {error_code} - {error_message}")
    except Exception as e:
        st.error(f"Unexpected error: {str(e)}")

    # Print debugging information
    print("\nDebugging Information:")
    print(f"AWS Region: {bedrock_runtime_client.meta.region_name}")
    print(f"Model used: {model_id}")
    print("Please ensure you have the correct permissions and that this model is available in your region.")

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501